# 湿空气焓湿图查询

原理：利用已有的免费在线查询网站构件 `API`。输入，输出到文件供VBA调用。

在线查询网址：http://www.buildenvi.com/gongju/psychrometrics#inputform

In [5]:
import math

In [18]:
def cal_pvs(t):
    '''
    输入：干球
    输出：饱和水蒸气分压力
    '''
    
    c1 = - 5674.5359
    c2 = 6.3925247
    c3 = - 0.009677843
    c4 = 6.2215701e-7
    c5 = 2.0747825e-9
    c6 = - 9.484024e-13
    c7 = 4.1635019
    c8 = - 5800.2206
    c9 = 1.3914993
    c10 = - 0.048640239
    c11 = 0.000041764768
    c12 = - 1.4452093e-8
    c13 = 6.5459673
    ta=t+273.15
    
    if t<0:
        pvs=0.001*math.exp(c1/ta +c2 +c3*ta +c4*ta**2 + c5*ta**3 + c6*ta**4 + c7*math.log(ta))
    else:
        pvs = 0.001*math.exp(c8/ta + c9 +c10*ta + c11*ta**2 + c12*ta**3 + c13*math.log(ta))
    
    return pvs

In [19]:
def cal_pv(t,tw,atm):
    '''
    输入：干球/湿球/大气压
    输出：水蒸气分压力
    '''
    
    pvs=cal_pvs(tw)   
    ds=(pvs/(atm-pvs))*0.621945
    
    if tw<=0:
        d = ((2830 - 0.24 * tw) * ds - 1.006 * (t - tw)) / (2830 + 1.86 * t - 2.1 * tw)
        pv = atm * (d / (0.621945 + d))
    else:
        d = ((2501 - 2.326 * tw) * ds - 1.006 * (t - tw)) / (2501 + 1.86 * t - 4.186 * tw)
        pv = atm * (d / (0.621945 + d))
    
    return pv

In [24]:
def cal_td(pv):
    
    '''
    Input: 水蒸气分压力
    Output: 露点温度
    '''
    
    y=math.log(pv)
    td = 6.09 + 12.608 * y + 0.4959 * y**2
    if td>=0:
        td = 6.54 + 14.526 * y + 0.7389 * y**2 + 0.09486 * y**3 + 0.4569 * pv**0.1984
    
    return td

参数名称：
- `tw`: wet bulb
- `rh`: relat_hum，相对湿度
- `td`: 露点温度
- `d`: 含湿量
- `h`: 比焓
- `pv`: 水蒸气分压力
- `pvs`： 饱和水蒸气分压力

In [29]:
def cal_d(t,tw,atm):
    
    '''
    Input: 干球/湿球/水蒸气分压力
    Output：含湿量（kg/kg）
    '''
    
    pv=cal_pv(t,tw,atm)
    d = 0.621945 * pv / (atm - pv)
    
    return d    

In [30]:
def cal_d_pv(atm,pv):
    
    '''
    Input: 大气压、水蒸气分压力
    Output：含湿量（kg/kg）
    '''
    
    d_pv = 0.621945 * pv / (atm - pv)
    return d_pv

In [34]:
def cal_h(t,tw,atm):
    
    '''比焓'''
    h = 1.006 * t + cal_d(t, tw, atm) * (2501 + 1.86 * t)
    return h

def cal_h_d(t, d):
    return  1.006 * t + d * (2501 + 1.86 * t)

In [37]:
def cal_rh(t, tw, atm):
    if t!=tw :
        rh= 100 * cal_pv(t, tw, atm) / cal_pvs(t)
    else:
        rh=100
    
    return rh        

In [38]:
def cal_v(t, tw, atm):
    
    '''比容'''
    return 0.287042 * (t + 273.15) * (1 + 1.607858 * cal_d(t, tw, atm)) / atm
    

In [50]:
def cal_tw(h, atm):
    acc=0.0001
    tlow=-100
    thigh=h/1.006
    
    for j in range(1,100):
        dt = (thigh - tlow) * 0.5
        tw = tlow + dt
        pvs = cal_pvs(tw)
        ds = cal_d_pv(atm, pvs)
        hmid = cal_h_d(tw, ds)
    
        if (h-hmid)>0:
            tlow=tw
        else:
            thigh=tw
        if abs(dt)<acc :
            break
    
    return tw
